In [1]:
import numpy
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import numpy as np

In [2]:
# Load the observed dataset
input_data = pd.read_csv('RL_input.csv')
ob_data = input_data.groupby('ObservedID')

In [3]:
# Link 
link = {1: [1, 3],
        2: [2, 5],
        3: [3, 4], 
        4: [5, 8],
        5: [4, 9],
        6: [7, 10],
        7: [8, 11],
        8: [9, 12],
        9: [10, 13],
       10: [11, 14],
       11: [12, 15],
       12: [3, 6],
       13: [5, 4],
       14: [4, 7],
       15: [8, 9],
       16: [9, 10],
       17: [11, 12],
       18: [12, 13],
       19: [11, 15],
       20: [14, 15],
       21: [6, 16],
       22: [7, 16],
       23: [16, 10],
       24: [16, 17],
       25: [16, 17],
       26: [10, 17],
       27: [13, 17],
       28: [12, 17],
       29: [15, 17]}

# network nodes
node = {17: tf.Variable([0.0, ], dtype=tf.float32), 
        16: tf.Variable([0.0, ], dtype=tf.float32), 
        15: tf.Variable([0.0, ], dtype=tf.float32), 
        14: tf.Variable([0.0, ], dtype=tf.float32),    
        13: tf.Variable([0.0, ], dtype=tf.float32), 
        12: tf.Variable([0.0, ], dtype=tf.float32), 
        11: tf.Variable([0.0, ], dtype=tf.float32), 
        10: tf.Variable([0.0, ], dtype=tf.float32),    
         9: tf.Variable([0.0, ], dtype=tf.float32), 
         8: tf.Variable([0.0, ], dtype=tf.float32), 
         7: tf.Variable([0.0, ], dtype=tf.float32), 
         6: tf.Variable([0.0, ], dtype=tf.float32),   
         5: tf.Variable([0.0, ], dtype=tf.float32), 
         4: tf.Variable([0.0, ], dtype=tf.float32), 
         3: tf.Variable([0.0, ], dtype=tf.float32), 
         2: tf.Variable([0.0, ], dtype=tf.float32), 
         1: tf.Variable([0.0, ], dtype=tf.float32)}

# Define Parameters (initial values)
travel_time = tf.Variable([-1.5, ], dtype=tf.float32, name='travel_time')
travel_sign = tf.Variable([-1.5,], dtype=tf.float32, name='travel_signal')
link_const = tf.Variable([-1.5, ], dtype=tf.float32, name='travel_constant')

param_val = [travel_time, travel_sign, link_const]

# Three attributes (travel time, travel_sign, link_const)
utility = {
            1:  [1, 0, 1],
            2:  [1, 0, 1],
            3:  [1.5, 1, 1],
            4:  [1.2, 0, 1],
            5:  [1.2, 1, 1],
            6:  [1.2, 1, 1],
            7:  [1.1,0, 1],
            8:  [1,  1, 1],
            9:  [1.2, 0, 1],
           10:  [0.5, 0, 1],
           11:  [0.5, 0, 1],
           12:  [1, 0, 1],
           13:  [1, 1, 1],
           14:  [1, 1, 1],
           15:  [1, 1, 1],
           16:  [1, 1, 1],
           17:  [0.8, 1,1],
           18:  [1.4, 0,1],
           19:  [0.9, 0,1],
           20:  [0.8, 0,1],
           21:  [2.1, 0,1],
           22:  [0.9, 0,1],
           23:  [1.7, 0, 1],
           24:  [5.3, 0, 1],
           25:  [4.5, 0, 1],
           26:  [3, 0, 1],
           27:  [2.5, 0, 1],
           28:  [3.3, 0, 1],
           29:  [3, 0, 1]
}

In [4]:
def model_fun(node, link, utility, param_val):
    
    # update the utility using three attributes with parameters
    up_utility = {}
    for p in range(len(utility)):
        
        up_utility[p+1] = param_val[0]*utility[p+1][0]+ param_val[1]*utility[p+1][1]+ param_val[2]*utility[p+1][2]
    
    # loop for computing the value functions 
    '''TO DO: optimize the calculation'''
    for i in node:
        # find the matched links
        a = [key for key, values in link.items() if link[key][0] == i]
        
        if a==[]:
            node[i] = tf.constant(0, dtype=tf.float32)

        if a!=[]:
            outnode = []
            sum_exp = tf.constant([0], dtype=tf.float32)
            for j in range(len(a)):
                # find the "tonode"
                outnode = np.append(outnode, link[a[j]][1])
                
                # calculate the exponential function
                exp_cal = tf.math.exp(tf.math.add(-up_utility[a[j]], node[outnode[j]]))
                exp_cal = tf.reshape(exp_cal, shape=[1, ])

                sum_exp = tf.concat([sum_exp, exp_cal], 0)

            node[i] = tf.math.log(tf.reduce_sum(sum_exp))
    
    up_node = node
    up_utility = up_utility
    
    link_prob = {}
    
    # Loop for calculating probabilities of each link
    for k in link:

        # find the matched links
        b = [key for key, values in link.items() if link[key][0] == k]

        if b==[]:
            break
        outnode = []
        exp_inv = tf.constant([0], dtype=tf.float32)

        # To compute each exponential function
        for p in range(len(b)):
            # find the "tonode"
            outnode = np.append(outnode, link[b[p]][1])
            # calculate the exponential function
            exp_cal = tf.math.exp(tf.math.add(-up_utility[b[p]], up_node[outnode[p]]))
            exp_cal = tf.reshape(exp_cal, shape=[1, ])
            exp_inv = tf.concat([exp_inv, exp_cal], 0)

        # To store probabilities of each link
        for t in range(len(b)):

            link_prob[b[t]] = exp_inv[t+1]/(tf.reduce_sum(exp_inv)) 
            
    return link_prob

def cost_func(ob_data, yhat):
    
    total_choice = tf.constant([0], dtype=tf.float32)

    for i in range(len(ob_data)):

        obs = ob_data.get_group(i+1).sequence.values

        path_choice = tf.constant([1], dtype=tf.float32)
        
        for h in obs:

            path_choice = tf.math.multiply(yhat[h], path_choice)

        total_choice = tf.concat([total_choice, path_choice], 0)
        
    est_cost = total_choice[1:]
    
    return -tf.reduce_sum(tf.math.log(est_cost))

In [5]:
# obtain the shapes of all trainable parameters in the model
def LL_gradient(node, link, utility, ob_data, param_val):
    model_fun(node, link, utility, param_val)
    shapes = tf.shape_n(param_val)
    n_tensors = len(shapes)
    
    count = 0
    idx =  [] 
    part = []
    
    for i, shape in enumerate(shapes):
        n = numpy.product(shape)
        idx.append(tf.reshape(tf.range(count, count+n, dtype=tf.int32), shape))
        part.extend([i]*n)
        count += n
        
    @tf.function
    def assign_new_model_parameters(params_1d):

        updated_params = tf.dynamic_partition(params_1d, part, n_tensors)
        
        for i, (shape, param) in enumerate(zip(shapes, updated_params)):
            param_val[i].assign(tf.reshape(param, shape))
                   
    @tf.function 
    def est_grad(params_1d):
        
        # Derive the Tensorflow gradient
        with tf.GradientTape(persistent=True) as tape: 
            
            # Call the function to update and convert the shape of parameters
            
            assign_new_model_parameters(params_1d)
            
            # Call the cost function
            
            yhat = model_fun(node, link, utility, param_val)
            
            loss_value = cost_func(ob_data, yhat)
            
        estimated_grad = tape.gradient(loss_value, param_val)
        grads_1dim = tf.dynamic_stitch(idx, estimated_grad)
        return loss_value, grads_1dim
    
    est_grad.idx = idx
    
    return est_grad

# Define the positions of initial parameters
init_params = tf.dynamic_stitch(LL_gradient(node, link, utility, ob_data, param_val).idx, param_val)

# Implement the BFGS optimizer
Trained_Results = tfp.optimizer.bfgs_minimize(
                                      value_and_gradients_function=LL_gradient(node, link, utility, ob_data, param_val), 
                                      initial_position=init_params,
                                      tolerance=1e-08,
                                      max_iterations=500)

In [8]:
param_val

[<tf.Variable 'travel_time:0' shape=(1,) dtype=float32, numpy=array([16.659325], dtype=float32)>,
 <tf.Variable 'travel_signal:0' shape=(1,) dtype=float32, numpy=array([48.786137], dtype=float32)>]